In [72]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import keras
from keras.models import Sequential
from keras.layers import *
from keras_preprocessing.image import ImageDataGenerator

import zipfile 

import cv2
import seaborn as sns
%matplotlib inline

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix

from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Model

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
%cd 'drive/MyDrive/Bangkit Capstone Project Team/Notebooks/face_data'

[Errno 2] No such file or directory: 'drive/MyDrive/Bangkit Capstone Project Team/Notebooks/face_data'
/content/drive/.shortcut-targets-by-id/1ZonyDXIOaWSJ6augTpgjNkWXh8rvEYl-/Bangkit Capstone Project Team/Notebooks/face_data


In [79]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime
import matplotlib.pyplot as plt

In [80]:
train_dir = 'train/'
test_dir = 'test/'

row, col = 48, 48
classes = 7

In [81]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [94]:
emotions

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [82]:
emotions = training_set.class_indices

In [83]:
model = tf.keras.models.Sequential()   

model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape =(48,48,3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax'))

#Compliling the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [105]:
hist = model.fit(x=training_set,
                 validation_data=test_set,
                 epochs=40,
                 steps_per_epoch=300)

Epoch 1/40
300/300 [==============================] - 44s 146ms/step - loss: 1.7760 - accuracy: 0.2751 - val_loss: 2.5689 - val_accuracy: 0.2216
Epoch 2/40
300/300 [==============================] - 43s 144ms/step - loss: 1.7354 - accuracy: 0.2925 - val_loss: 1.7170 - val_accuracy: 0.3236
Epoch 3/40
300/300 [==============================] - 42s 141ms/step - loss: 1.7078 - accuracy: 0.3145 - val_loss: 1.7442 - val_accuracy: 0.2868
Epoch 4/40
300/300 [==============================] - 43s 145ms/step - loss: 1.6614 - accuracy: 0.3327 - val_loss: 1.6227 - val_accuracy: 0.3837
Epoch 5/40
300/300 [==============================] - 42s 141ms/step - loss: 1.6309 - accuracy: 0.3513 - val_loss: 1.5991 - val_accuracy: 0.3409
Epoch 6/40
300/300 [==============================] - 42s 140ms/step - loss: 1.5980 - accuracy: 0.3688 - val_loss: 1.5769 - val_accuracy: 0.4051
Epoch 7/40
300/300 [==============================] - 43s 142ms/step - loss: 1.5717 - accuracy: 0.3831 - val_loss: 1.5810 - val_ac

In [108]:
model.evaluate(training_set)

449/449 [==============================] - 47s 105ms/step - loss: 1.1960 - accuracy: 0.5688


[1.196007251739502, 0.5688111782073975]

In [107]:
model.evaluate(test_set)

113/113 [==============================] - 8s 66ms/step - loss: 1.4197 - accuracy: 0.5464


[1.4197286367416382, 0.5463917255401611]

In [109]:
model.save('face_model')

INFO:tensorflow:Assets written to: face_model/assets


INFO:tensorflow:Assets written to: face_model/assets


In [111]:
converter = tf.lite.TFLiteConverter.from_saved_model('face_model')
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

In [112]:
tflite_model_file = 'face_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

In [113]:
# Load TFLite model and allocate tensors.
with open(tflite_model_file, 'rb') as fid:
    tflite_model = fid.read()
    
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [170]:
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload().keys()
for path in uploaded:
  img_3 = image.load_img(path, target_size=(48,48))
  x = image.img_to_array(img_3)
  x = np.expand_dims(x, axis=0)
  imagess = np.vstack([x])
  # classes = loaded.predict(imagess)
  # print(classes)
  # print(emotions[classes.argmax()])

Saving images (1).jpg to images (1) (2).jpg


In [171]:
ii = tf.convert_to_tensor(imagess, dtype=tf.float32)

In [172]:
ii.shape

TensorShape([1, 48, 48, 3])

In [173]:
# Gather results for the randomly sampled test images
predictions = []

test_lbls, test_imgs = [], []
interpreter.set_tensor(input_index, ii)
interpreter.invoke()
predictions.append(interpreter.get_tensor(output_index))
  
  # test_lbls.append(test_labels[i])
  # test_imgs.append(img)

In [174]:
predictions

[array([[0., 0., 0., 1., 0., 0., 0.]], dtype=float32)]